# Imbalanced classification: credit card fraud detection

## importing libraries

In [2]:
import csv
import numpy as np
from tensorflow import keras

## Import data

In [6]:
fname = r"C:\Users\dell\Downloads\creditcard.csv"

In [7]:
all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])
#array lists of features and target 
features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


### Prepare a validation set


In [8]:
num_val_samples = int(len(features) * 0.2)
#80% of data is for training set
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


### Analyze class imbalance in the targets

In [9]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


### Normalize the data using training set statistics

In [10]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

## Build a binary classification model

In [11]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7936      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 139,777
Trainable params: 139,777
Non-trai

## Train the model with class_weight argument

In [12]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)
    
callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/30
112/112 - 4s - loss: 2.2744e-06 - fn: 39.0000 - fp: 27395.0000 - tn: 200034.0000 - tp: 378.0000 - precision: 0.0136 - recall: 0.9065 - val_loss: 0.0927 - val_fn: 8.0000 - val_fp: 1067.0000 - val_tn: 55819.0000 - val_tp: 67.0000 - val_precision: 0.0591 - val_recall: 0.8933 - 4s/epoch - 38ms/step
Epoch 2/30
112/112 - 3s - loss: 1.6079e-06 - fn: 36.0000 - fp: 8764.0000 - tn: 218665.0000 - tp: 381.0000 - precision: 0.0417 - recall: 0.9137 - val_loss: 0.1085 - val_fn: 11.0000 - val_fp: 457.0000 - val_tn: 56429.0000 - val_tp: 64.0000 - val_precision: 0.1228 - val_recall: 0.8533 - 3s/epoch - 26ms/step
Epoch 3/30
112/112 - 3s - loss: 1.4376e-06 - fn: 34.0000 - fp: 8434.0000 - tn: 218995.0000 - tp: 383.0000 - precision: 0.0434 - recall: 0.9185 - val_loss: 0.0811 - val_fn: 9.0000 - val_fp: 794.0000 - val_tn: 56092.0000 - val_tp: 66.0000 - val_precision: 0.0767 - val_recall: 0.8800 - 3s/epoch - 25ms/step
Epoch 4/30
112/112 - 3s - loss: 1.0207e-06 - fn: 25.0000 - fp: 5888.0000 - tn: 221

Epoch 28/30
112/112 - 3s - loss: 9.2349e-07 - fn: 10.0000 - fp: 5480.0000 - tn: 221949.0000 - tp: 407.0000 - precision: 0.0691 - recall: 0.9760 - val_loss: 0.0527 - val_fn: 9.0000 - val_fp: 890.0000 - val_tn: 55996.0000 - val_tp: 66.0000 - val_precision: 0.0690 - val_recall: 0.8800 - 3s/epoch - 29ms/step
Epoch 29/30
112/112 - 3s - loss: 4.0395e-07 - fn: 4.0000 - fp: 3547.0000 - tn: 223882.0000 - tp: 413.0000 - precision: 0.1043 - recall: 0.9904 - val_loss: 0.0388 - val_fn: 9.0000 - val_fp: 853.0000 - val_tn: 56033.0000 - val_tp: 66.0000 - val_precision: 0.0718 - val_recall: 0.8800 - 3s/epoch - 30ms/step
Epoch 30/30
112/112 - 4s - loss: 3.5620e-07 - fn: 2.0000 - fp: 3555.0000 - tn: 223874.0000 - tp: 415.0000 - precision: 0.1045 - recall: 0.9952 - val_loss: 0.0110 - val_fn: 12.0000 - val_fp: 153.0000 - val_tn: 56733.0000 - val_tp: 63.0000 - val_precision: 0.2917 - val_recall: 0.8400 - 4s/epoch - 37ms/step
